<img src="https://www.google.com/url?sa=i&url=https%3A%2F%2Faws.amazon.com%2Fes%2Fsolutions%2Fcase-studies%2FKueski%2F&psig=AOvVaw2G4P4XL7b6yQb4dv5CCv6i&ust=1640206653391000&source=images&cd=vfe&ved=0CAsQjRxqFwoTCOjUsInk9fQCFQAAAAAdAAAAABAD">


## Glue Job Preprocessing with PySpark- Model training-Kueski Challenge

AWS Glue supports an extension of the PySpark Python dialect for scripting extract, transform, and load (ETL) jobs. In this notebook we use this dialect for creating an ETL script to run a Glue job. 

<a id='contents' />

## Table of contents

1. [Loading libraries](#loading)
2. [Data Preprocessing. Data Pipelines](#etl)
3. [Model Training.](#s3)

<a id='loading' />

## 1. Loading libraries:
[(back to top)](#contents)

In [1]:
print('Loading the required libraries')
import sys
import pyspark.sql.functions as func
from awsglue.dynamicframe import DynamicFrame
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.sql.types import *
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql.functions import *
from pyspark.sql import Window
import json
import boto3
import ast
import datetime
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import gc
import sys
from pyspark.conf import SparkConf
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
import pandas as pd

kms_key_id = '36f1041a-656a-4df4-954a-49b6a39e4b54'

spark_conf = SparkConf().setAll([
    ("spark.hadoop.fs.s3.enableServerSideEncryption", "true"),
    ("spark.hadoop.fs.s3.serverSideEncryption.kms.keyId", kms_key_id),
    ("spark.sql.sources.partitionOverwriteMode", "dynamic"),
    ("hive.metastore.client.factory.class", "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory"),
    ("spark.sql.catalogImplementation", "hive"),
    ("hive.exec.dynamic.partition.mode", "nonstrict")
])
sc = SparkContext.getOrCreate(conf=spark_conf)
glueContext = GlueContext(sc)
spark = glueContext.spark_session
logger = glueContext.get_logger()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
165,application_1622637872219_0165,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Loading the required libraries

In [21]:
# Dejar variables del processing_job en el proceso.
# Definir variable updated_at. ( podria ser una particion para quedarme solo con aquellas particiones que son las ultimas)
# Primary key deberia ser el loan_id. Indice sobre el id del cliente.
today       = datetime.strptime("2021-10-21", '%Y-%m-%d').date()
updated_at  = today.strftime('%Y-%m-%d')
source_path = "s3://arcosmtk-working-directory/credits_scoring_test/offline_serving/"
# In order to evaluate the performance (backtesting) to measure model degradation. e.g. KS score, AUC or Recall
# Date of the simulated data collection.
ObservationDate   = "2020-12-21"
ObservationPeriod = 12
PerformancePeriod = 12
#TODO create routines to measure degradation.

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# Esto lo deberia leer desde el catalogo. Control de columnas y permisos.GDPR
df = glueContext.read.parquet(source_path)\
    .select('id','nb_previous_loans','avg_amount_loans_previous','age','years_on_the_job','flag_own_car','status','loan_date')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# Skipped step in the original training notebook. Pseudo-replicas problem. Id must be unique. 
df = df.orderBy(col("id").asc(),col("loan_date").desc()).dropDuplicates(['id'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<a id='etl' />

## Data Preprocessing. Data Pipelines.
[(back to top)](#contents)

In [26]:
# Value counts
vc =  df.groupby('status').count()
vc = vc.withColumn('freq', col('count')/float(df.count()))
vc.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
# Keep just selected features by DS.
df = df.select('nb_previous_loans','avg_amount_loans_previous','age','years_on_the_job','flag_own_car','status')
# fillna #TODO improved by Pipeline.
df = df.na.fill(0)
# setting X,y.
X = df.select('nb_previous_loans','avg_amount_loans_previous','age','years_on_the_job','flag_own_car')
y = df.select('status')
del df
gc.collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<a id='etl' />

## Model Training.
[(back to top)](#contents)